In [ ]:
from astropy.io import fits
import clevar
import pandas as pd
from astropy.table import Table
import astropy
from matplotlib import pyplot as plt
import numpy as np
from numcosmo_py import Ncm, Nc
from numcosmo_py.external.pyssc import pyssc as PySSC
Ncm.cfg_init()

In [ ]:
hdul_halos = fits.open("../halos/halos.fits")
hdul_amico = fits.open("JPAS_mock_cut_map_detections_refined_amico.fits")

table_halos = hdul_halos[1].data
table_amico = hdul_amico[1].data

logMmin = 13
logMmax = 15
table_amico.columns["Xphys"].name = 'RA'
table_amico.columns["Yphys"].name = 'DEC'
table_amico.columns["Zphys"].name = 'z'

table_halos = table_halos[(table_halos['upid'] == -1)  &  (table_halos['Mvir'] >= 10**logMmin) & (table_halos['Mvir']
             <= 10**logMmax) & (table_halos['DEC'] <=table_amico['DEC'].max()) & (table_halos['DEC'] >= table_amico['DEC'].min())
                      &   (table_halos['RA'] <= table_amico['RA'].max()) & (table_halos['RA'] >=table_amico['RA'].min())]
table_halos['Mvir'] = table_halos['Mvir']/0.6774

print(table_halos.size)
print(table_amico.size)


#print(table_amico.columns)

In [ ]:
catalog_halos = clevar.ClCatalog('Halo catalog',ra=table_halos['RA'] ,dec=table_halos['DEC'] 
                                ,z=table_halos['redshift_R'], mass=table_halos['Mvir'])
catalog_amico = clevar.ClCatalog('Detection catalog',SN=table_amico['SN'], ra=table_amico['RA'], dec=table_amico['DEC'], mass=table_amico['LAMBSTAR'],z=table_amico['z'] )

In [ ]:
mt = clevar.match.ProximityMatch()

match_config = {
    'type': 'cross', # options are cross, cat1, cat2
    'which_radius': 'max', # Case of radius to be used, can be: cat1, cat2, min, max
    'preference': 'redshift_proximity', # options are more_massive, angular_proximity or redshift_proximity
    'catalog1': {'delta_z':0.01,
                'match_radius': '1000 Mpc'
                },
    'catalog2': {'delta_z':0.01,
                'match_radius': '1000 Mpc'
                }
}

cosmo_astropy = clevar.cosmology.AstroPyCosmology()

catalog_halos = clevar.ClCatalog('Halo catalog',ra=table_halos['RA'] ,dec=table_halos['DEC'] 
                                ,z=table_halos['redshift_R'], mass=table_halos['Mvir'])

catalog_amico = clevar.ClCatalog('Detection catalog',SN=table_amico['SN'], ra=table_amico['RA'], dec=table_amico['DEC'], mass=table_amico['LAMBSTAR'],z=table_amico['z'] )
mt = clevar.match.ProximityMatch()
mt.load_matches(catalog_halos, catalog_amico, out_dir='temp_lambstar')

halos_match, detections_match = clevar.match.get_matched_pairs(catalog_halos, catalog_amico, 'cross')

In [ ]:
pz_0 = []
z = np.linspace(0,0.7 , 180)
for i in range(180):
    pz_0.append(table_amico['Pz-%i' %(i)].values[0])

plt.plot(z , pz_0)

pz_46607 = []
z = np.linspace(0,0.7 , 180)
for i in range(180):
    pz_46607.append(table_amico['Pz-%i' %(i)].values[46607])

plt.plot(z , pz_46607)


pz_55770 = []
z = np.linspace(0,0.7 , 180)
for i in range(180):
    pz_55770.append(table_amico['Pz-%i' %(i)].values[55770])

plt.plot(z , pz_55770)

plt.show()

In [ ]:
area = 4109.3
cosmo = Nc.HICosmoDEXcdm()
cosmo.omega_x2omega_k()
cosmo.param_set_by_name("H0", 67.74)
cosmo.param_set_by_name("Omegab", 0.0486)
cosmo.param_set_by_name("Omegak", 0.00)
cosmo.param_set_by_name("Omegac", 0.2603)
cosmo.param_set_by_name("w", -1.0)


prim = Nc.HIPrimPowerLaw.new()
prim.param_set_by_name("ln10e10ASA", 3.02745)
prim.param_set_by_name("n_SA", 0.9667)

reion = Nc.HIReionCamb.new()

cosmo.add_submodel(prim)
cosmo.add_submodel(reion)

dist = Nc.Distance.new(2.0)

tf = Nc.TransferFuncEH()

psml = Nc.PowspecMLTransfer.new(tf)
psml.require_kmin(1.0e-6)
psml.require_kmax(1.0e3)

psf = Ncm.PowspecFilter.new(psml, Ncm.PowspecFilterType.TOPHAT)
psf.set_best_lnr0()

mulf = Nc.MultiplicityFuncTinker.new()
mulf.set_mdef(Nc.MultiplicityFuncMassDef.CRITICAL)
mulf.set_Delta(200.0)

hmf = Nc.HaloMassFunction.new(dist, psf, mulf)
hmf.set_area(area * (np.pi / 180) ** 2)
hmf.prepare(cosmo)
hbias_Tinker = Nc.HaloBiasTinker.new(hmf)
cad = Nc.ClusterAbundance.new(hmf, hbias_Tinker)
cad.set_area(area * (np.pi / 180) ** 2)
hmf.prepare(cosmo)

#σ_8 = 0.8159

In [ ]:
#mass proxy
cluster_m_lamb = Nc.ClusterMassAscaso(lnRichness_min = 0.0 ,lnRichness_max = np.log(60.0))
cluster_m_lamb.param_set_by_name("mup0", 4.9)
cluster_m_lamb.param_set_by_name("mup1", 0.993)
cluster_m_lamb.param_set_by_name("mup2", 0)
cluster_m_lamb.param_set_by_name("sigmap0", 0.26)
cluster_m_lamb.param_set_by_name("sigmap1", 0.0)
cluster_m_lamb.param_set_by_name("sigmap2", 0.0)

cluster_m_lamb.set_property("mup0_fit", True)
cluster_m_lamb.set_property("mup1_fit", True)
cluster_m_lamb.set_property("sigmap0_fit", True)

cluster_m_lambstar = Nc.ClusterMassAscaso(lnRichness_min = 0.0 ,lnRichness_max = np.log(25))
cluster_m_lambstar.param_set_by_name("mup0", 3.807)
cluster_m_lambstar.param_set_by_name("mup1", 0.993)
cluster_m_lambstar.param_set_by_name("mup2", 0)
cluster_m_lambstar.param_set_by_name("sigmap0", 0.2)
cluster_m_lambstar.param_set_by_name("sigmap1", 0.0)
cluster_m_lambstar.param_set_by_name("sigmap2", 0.0)

cluster_m_lambstar.set_property("mup0_fit", True)
cluster_m_lambstar.set_property("mup1_fit", True)
cluster_m_lambstar.set_property("sigmap0_fit", True)


#photo-z
cluster_z = Nc.ClusterRedshiftNodist (z_min = 0.0 , z_max = 0.7)

#mass-proxy plots
masses_high_lamb  = np.linspace(np.log(10) *14.6 , np.log(10) * 16.5,10000)
masses_10_high_lamb  = np.linspace(13.75 ,14.6,10000)
relation_high_lamb = []

masses_low_lamb  = np.linspace(np.log(10) *12 , np.log(10) * 13.0,10000)
masses_10_low_lamb  = np.linspace(12.0 ,14.0,10000)
relation_low_lamb = []

masses_high_lambstar  = np.linspace(np.log(10) *14.5 , np.log(10) * 15.5,10000)
masses_10_high_lambstar  = np.linspace(14.5 ,16.5,10000)
relation_high_lambstar = []

masses_low_lambstar  = np.linspace(np.log(10) *12 , np.log(10) * 14.0,10000)
masses_10_low_lambstar  = np.linspace(12.0 ,14.0,10000)
relation_low_lambstar = []

for i in range(len(masses_high_lamb)):
    relation_high_lamb.append(cluster_m_lamb.p(cosmo , masses_high_lamb[i], 0.1 , [np.log(470.350090)] , None))
    relation_low_lamb.append(cluster_m_lamb.p(cosmo , masses_low_lamb[i], 0.1 , [np.log(1.082300)] , None))

    relation_high_lambstar.append(cluster_m_lambstar.p(cosmo , masses_high_lambstar[i], 0.1 , [np.log(119.153600)] , None))
    relation_low_lambstar.append(cluster_m_lambstar.p(cosmo , masses_low_lambstar[i], 0.1 , [np.log(0.795985)] , None))

plt.figure(figsize = (15,10))

plt.subplot(2, 2, 1)
plt.plot(masses_10_high_lamb , relation_high_lamb , label = r"$\lambda = %.2f$" %(470.350090))
plt.ylabel(r'$P(ln\lambda|M,z)$')
plt.xlabel(r'$lnM$')
plt.legend()


plt.subplot(2, 2, 2)
plt.plot(masses_10_low_lamb , relation_low_lamb , label = r"$\lambda = %.2f$" %(1.082300))
plt.xlabel(r'$lnM$')
plt.legend()


plt.subplot(2, 2, 3)
plt.plot(masses_10_high_lambstar , relation_high_lambstar , label = r"$\lambda_s = %.2f$" %(119.153600))
plt.ylabel(r'$P(ln\lambda_s|M,z)$')
plt.xlabel(r'$lnM$')
plt.legend()


plt.subplot(2, 2, 4)
plt.plot(masses_10_low_lambstar , relation_low_lambstar , label = r"$\lambda_s = %.2f$" %(0.795985))
plt.xlabel(r'$lnM$')
plt.legend()

plt.show()

In [ ]:
multiplicity = [Nc.MultiplicityFuncST.new() , Nc.MultiplicityFuncJenkins.new() , Nc.MultiplicityFuncBocquet.new()
               , Nc.MultiplicityFuncWatson.new() , Nc.MultiplicityFuncTinker.new() , Nc.MultiplicityFuncDespali.new()] 
legend = ['ST'  , 'Jenkins' , 'Bocquet' , "Watson" , 'Tinker' , 'Despali']

for mult in range(len(multiplicity)):

    richness = np.linspace(np.log(1) , np.log(100.0), 100)
    richness_plot = []
    n_bin = []
    
    mulf = multiplicity[mult]
    if mult == 4:
        mulf.set_mdef(Nc.MultiplicityFuncMassDef.CRITICAL)
        mulf.set_Delta(200.0)
    if mult == 5:
        mulf.set_mdef(Nc.MultiplicityFuncMassDef.VIRIAL)

    hmf = Nc.HaloMassFunction.new(dist, psf, mulf)
    hmf.set_area(area * (np.pi / 180) ** 2)
    hmf.prepare(cosmo)
    cad = Nc.ClusterAbundance.new(hmf, None)
    cad.set_area(area * (np.pi / 180) ** 2)
    hmf.prepare(cosmo)

    for i in range(len(richness)-1):
        n_bin.append(cad.intp_bin_d2n(cosmo , cluster_z , cluster_m_lamb , [richness[i]] ,[richness[i+1]],None ,[0.0] , [0.7] , None))
        richness_plot.append(np.exp((richness[i] + richness[i+1])/2))
    
    
    
    
    plt.plot(np.log10(richness_plot) , np.log10(n_bin) , label = "%s" %(legend[mult]))
    
    
    richness_plot = []
    n_bin = []
    richness = np.linspace(np.log(1) , np.log(40) , 1000)
    
    cad.prepare(cosmo , cluster_z , cluster_m_lambstar)
    cad.set_area(area * (np.pi / 180) ** 2)

plt.legend()
plt.hist(np.log(table_amico['LAMB']) , bins=len(richness))
plt.yscale("log")
plt.show()
'''
for i in range(len(richness)-1):
    n_bin.append(cad.intp_bin_d2n(cosmo , cluster_z , cluster_m_lambstar , [richness[i]] ,[richness[i+1]],None ,[0] , [0.7] , None))
    richness_plot.append(np.exp((richness[i] + richness[i+1])/2))

'''

print(table_amico['LAMBSTAR'])
plt.xlim(0 , 4.5)
plt.hist(np.log(table_amico['LAMBSTAR']) , bins=len(richness)-1)
plt.plot(np.log(richness_plot) , np.log10(n_bin))
plt.yscale("log")
plt.show()


In [ ]:
z = np.linspace(0.0,0.7 , 180)
z_array = []


for j in range(len(table_amico['ID'])):
    pz = []
    for i in range(180):
        pz.append(table_amico['Pz-%i' %(i)].values[j])
    z_array.append(z[pz.index(max(pz))])

z = np.linspace(0 , 0.7, 1000)
z_plot = []
n_bin = []

for i in range(len(z)-1):
        n_bin.append(cad.intp_bin_d2n(cosmo , cluster_z , cluster_m_lamb , [1] ,[np.log(60)],None ,[z[i]] , [z[i+1]] , None))
        z_plot.append((z[i] + z[i+1])/2)

plt.hist(z_array , bins= 50)
plt.plot(z_plot , n_bin)
plt.show()